In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 30
LAYER_SIZE = 30
S_LAYER_SIZE = 15
LATENT_SIZE = 6

In [3]:
# Load the data from the file

saved = np.load("testing.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.set_printoptions(linewidth=140)

torch.Size([50000, 11])


In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, S_LAYER_SIZE)
        self.d1 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.d2 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l2a = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, S_LAYER_SIZE)
        self.d3 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.d4 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l4 = nn.Linear(S_LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        # return the mu and the sigma
        return self.l2a(x), self.l2b(x)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        x = F.relu(self.l3(x))
        x = F.relu(self.d3(x))
        x = F.relu(self.d4(x))
        return torch.sigmoid(self.l4(x)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
#     KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
#     return BCE + KLD
    return BCE

In [7]:
def train(e_count, model, optimizer):
    data = train_data
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
train_data=train_data[torch.randperm(train_data.size()[0])]
test_data=test_data[torch.randperm(test_data.size()[0])]

In [10]:
vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

In [11]:
for i in range(EPOCHS):
    train(i, vae1, opt1)

Epoch: 0, Loss: 6.738183416748047
Epoch: 1, Loss: 6.254246844482422
Epoch: 2, Loss: 6.2341465502929685
Epoch: 3, Loss: 6.23286374633789
Epoch: 4, Loss: 6.232385623168946
Epoch: 5, Loss: 6.232164191894531
Epoch: 6, Loss: 6.232026492919922
Epoch: 7, Loss: 6.231876740722656
Epoch: 8, Loss: 6.2317273309326175
Epoch: 9, Loss: 6.2315262127685545
Epoch: 10, Loss: 6.231237077026367
Epoch: 11, Loss: 6.230756421508789
Epoch: 12, Loss: 6.229849782714844
Epoch: 13, Loss: 6.227871381225586
Epoch: 14, Loss: 6.22282283203125
Epoch: 15, Loss: 6.213751964111328
Epoch: 16, Loss: 6.2076814221191405
Epoch: 17, Loss: 6.204412852172852
Epoch: 18, Loss: 6.203541481933594
Epoch: 19, Loss: 6.203331782226562
Epoch: 20, Loss: 6.203235104370117
Epoch: 21, Loss: 6.2031801879882815
Epoch: 22, Loss: 6.203138837890625
Epoch: 23, Loss: 6.203102510986328
Epoch: 24, Loss: 6.203069411010742
Epoch: 25, Loss: 6.203049979248047
Epoch: 26, Loss: 6.203027161865235
Epoch: 27, Loss: 6.203007424316406
Epoch: 28, Loss: 6.20298542

In [12]:
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_low = env.action_space.low[0]
action_high = env.action_space.high[0]
state_low = -10.0
state_high = 10.0

In [13]:
def descale(x):
    # State
    ((x[:, 0].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 1].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 2].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 3].mul_(state_high-state_low)).add_(state_low)).to(device)

    # Action
    ((x[:, 4].mul_(action_high-action_low)).add_(action_low)).to(device)
    
    # State
    ((x[:, 5].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 6].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 7].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 8].mul_(state_high-state_low)).add_(state_low)).to(device)
    
    # Reward
    (x[:, 9].mul_(20.0)).to(device)
    
    # Done
    (x[:, 10].round_()).to(device)
    
    return x

In [14]:
from torch.autograd import Variable

print("Action")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 1).to("cuda")
    print(sample[0][5])
    x = vae1(sample)
    print(x[0][0][5], '\n')

Action
tensor(0.0709, device='cuda:0')
tensor(0.5087, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8194, device='cuda:0')
tensor(0.5025, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8441, device='cuda:0')
tensor(0.4926, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1289, device='cuda:0')
tensor(0.5039, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8498, device='cuda:0')
tensor(0.4980, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1809, device='cuda:0')
tensor(0.4919, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.9225, device='cuda:0')
tensor(0.4694, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.6081, device='cuda:0')
tensor(0.5098, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4483, device='cuda:0')
tensor(0.5017, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.9408, device='cuda:0')
tensor(0.5024, device='cuda:0', grad_fn=<SelectBackward>) 



In [15]:
print("Reward")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 0.15).to("cuda")
    print(sample[0][9])
    x = vae1(sample)
    print(x[0][0][9], '\n')

Reward
tensor(0.1183, device='cuda:0')
tensor(0.0609, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0431, device='cuda:0')
tensor(0.0600, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0486, device='cuda:0')
tensor(0.0667, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0923, device='cuda:0')
tensor(0.0304, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1363, device='cuda:0')
tensor(0.0447, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0311, device='cuda:0')
tensor(0.0639, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1470, device='cuda:0')
tensor(0.0694, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1196, device='cuda:0')
tensor(0.0671, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0429, device='cuda:0')
tensor(0.0665, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0167, device='cuda:0')
tensor(0.0677, device='cuda:0', grad_fn=<SelectBackward>) 



In [22]:
torch.set_printoptions(precision=3, sci_mode=False, linewidth=140, profile=None)
import random
for i in range(5):
    k = random.randint(1, 400)
    sample = test_data[k].to(device)
    x, m, s = vae1(sample)
    loss = loss_function(x, sample, m, s)
    
    x = x.to("cpu")
    sample = sample.to("cpu")
    
    print(f"Original:       {sample}")
    sample = sample.to(device)
    print(f"Reconstruction: {x}\n")
    print(f"Latent: {list(vae1.encode(sample))[0]}")
    print(f"\nLoss: {loss}\n\n")
    

Original:       tensor([0.501, 0.499, 0.465, 0.576, 0.956, 0.501, 0.500, 0.515, 0.454, 0.050, 0.000])
Reconstruction: tensor([    0.500,     0.502,     0.464,     0.578,     0.907,     0.500,     0.500,     0.517,     0.457,     0.051,     0.000],
       grad_fn=<CopyBackwards>)

Latent: tensor([ 3.548, 10.166, -6.968, -2.254, -5.846,  6.463], device='cuda:0', grad_fn=<AddBackward0>)

Loss: 5.922659873962402


Original:       tensor([0.498, 0.505, 0.541, 0.410, 0.993, 0.501, 0.499, 0.594, 0.296, 0.050, 0.000])
Reconstruction: tensor([    0.501,     0.499,     0.536,     0.418,     0.921,     0.505,     0.492,     0.591,     0.300,     0.051,     0.000],
       grad_fn=<CopyBackwards>)

Latent: tensor([ 3.524,  9.495, -6.428, -2.631, -5.777,  6.409], device='cuda:0', grad_fn=<AddBackward0>)

Loss: 5.720589637756348


Original:       tensor([0.501, 0.497, 0.552, 0.378, 0.031, 0.502, 0.495, 0.500, 0.502, 0.050, 0.000])
Reconstruction: tensor([    0.503,     0.499,     0.552,     0.380,   

In [17]:
for i in range(99):
    print(train_data[i])

tensor([0.502, 0.494, 0.530, 0.425, 0.993, 0.505, 0.489, 0.583, 0.301, 0.050, 1.000])
tensor([0.502, 0.497, 0.539, 0.411, 0.140, 0.502, 0.496, 0.499, 0.505, 0.050, 0.000])
tensor([0.500, 0.500, 0.500, 0.500, 0.001, 0.499, 0.502, 0.445, 0.627, 0.050, 0.000])
tensor([0.502, 0.495, 0.515, 0.468, 0.998, 0.504, 0.491, 0.569, 0.339, 0.050, 0.000])
tensor([0.503, 0.494, 0.537, 0.424, 0.825, 0.506, 0.489, 0.572, 0.341, 0.050, 1.000])
tensor([0.500, 0.500, 0.500, 0.500, 0.101, 0.499, 0.502, 0.457, 0.602, 0.050, 0.000])
tensor([0.500, 0.500, 0.500, 0.500, 0.052, 0.499, 0.502, 0.450, 0.614, 0.050, 0.000])
tensor([0.501, 0.498, 0.480, 0.550, 0.625, 0.500, 0.499, 0.494, 0.513, 0.050, 0.000])
tensor([0.500, 0.500, 0.536, 0.415, 0.027, 0.501, 0.499, 0.483, 0.541, 0.050, 0.000])
tensor([0.501, 0.498, 0.569, 0.371, 0.762, 0.504, 0.492, 0.597, 0.311, 0.050, 0.000])
tensor([0.500, 0.500, 0.500, 0.500, 0.064, 0.499, 0.502, 0.452, 0.611, 0.050, 0.000])
tensor([0.500, 0.500, 0.500, 0.500, 0.623, 0.500, 0.49

In [18]:
vae1(test_data[0].to(device))

(tensor([    0.501,     0.500,     0.497,     0.504,     0.626,     0.501,     0.499,     0.510,     0.477,     0.051,     0.000],
        device='cuda:0', grad_fn=<SigmoidBackward>),
 tensor([ 2.789,  8.215, -5.806, -0.920, -4.158,  4.739], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([-14.816, -16.048, -12.929, -15.694, -13.095, -15.291], device='cuda:0', grad_fn=<AddBackward0>))